In [36]:
# Importing
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from keras.utils import to_categorical
import keras
from keras.layers import Dense
from keras import Sequential

In [2]:
data = pd.read_csv("matches.csv")

In [3]:
data.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [4]:
data.toss_decision = data.toss_decision.map({'bat':1, 'field':0})

In [5]:
data.result = data.result.map({'normal':1, 'tie':2, 'no result':0})

In [6]:
data.drop(columns=['venue', 'player_of_match', 'dl_applied','umpire1','umpire2','umpire3','date','city','season','id'], inplace=True)

In [7]:
r = len(data.team2.unique())
teams = data.team1.unique()
mapping = {}

In [8]:
for i in range(14): # There are 14 teams.
    mapping[teams[i]] = i

In [9]:
# Encoding data in numeric form
data.team1 = data.team1.map(mapping)
data.team2 = data.team2.map(mapping)

In [10]:
data.toss_winner = data.toss_winner.map(mapping)

In [11]:
data.winner = data.winner.map(mapping)

In [12]:
data.winner.fillna(0, axis=0, inplace=True)

In [13]:
data.winner = data.winner.astype(int)

In [14]:
for i in range(len(data['win_by_runs'].values)):
    if (data['win_by_runs'][i]) >= 20: # Strong team
        data['win_by_runs'][i] = 1
    else:
        data['win_by_runs'][i] = 0

In [15]:
for i in range(len(data['win_by_wickets'].values)):
    if data['win_by_wickets'][i] >= 7: # Strong team
        data['win_by_wickets'][i] = 1
    else:
        data['win_by_wickets'][i] = 0

In [16]:
data.head()

,team1,team2,toss_winner,toss_decision,result,winner,win_by_runs,win_by_wickets
0,0,4,4,0,1,0,1,0
1,1,3,3,0,1,3,0,1
2,2,5,5,0,1,5,0,1
3,3,7,7,0,1,7,0,0
4,4,6,4,1,1,4,0,0


In [17]:
# Extracting features and labels
labels = data.winner.values
features = data.drop(columns=["winner"]).values

In [18]:
# For neural network
ndim = features.shape[1]

In [19]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, random_state=3, shuffle=True)

In [20]:
len(features_train)

477

In [22]:
len(data.columns)

8

In [23]:
features.shape[1]

7

In [25]:
# Viewing labels
labels

array([ 0,  3,  5,  7,  4,  0,  1,  7,  6,  1,  5,  1,  2,  5,  6,  1,  3,
        5,  0,  4,  0,  1,  2,  1,  3,  7,  5,  3,  5,  2,  5,  0,  3,  1,
        7,  0,  1,  3,  6,  3,  6,  7,  3,  1,  5,  2,  0,  7,  6,  7,  6,
        0,  1,  3,  4,  3,  5,  1,  1,  5,  8,  6,  4,  5,  9,  6,  8,  9,
        7,  9,  8, 10,  7,  8,  1,  6,  7,  9,  6,  4,  7,  1,  9,  7, 10,
        1,  8,  5,  9,  1,  8,  5,  9,  7,  5,  1,  7,  6,  1,  7,  9,  1,
        8,  6,  9,  7,  4,  7,  6,  9,  4,  5,  9,  8,  9,  8,  9,  1,  4,
        6, 10,  8,  5, 10,  6,  9,  7, 10,  6,  7, 10,  1,  9,  4,  7,  6,
        8,  1,  4,  9,  8,  7,  4,  8,  9,  6, 10,  9,  8,  6,  7,  8,  1,
        6, 10,  4,  1,  6,  4,  9,  7,  8, 10,  7,  6,  5,  4,  5,  8,  6,
        4, 10,  4, 10,  5,  1,  6,  5, 10,  6,  4,  8,  1,  4,  8, 10,  9,
        4, 10,  7,  1,  4,  9,  1,  9,  5,  6,  9,  1,  6,  1,  8,  6,  5,
        4,  8,  1,  7,  6,  9,  8,  9,  5, 10,  7, 10,  4,  7,  1, 10,  1,
        8,  4,  6, 10,  1

In [26]:
# As the labels are multi class, we perform one hot encoding.

In [27]:
labels_copy = to_categorical(labels)

In [40]:
labels_copy.shape
features_copy_train, features_copy_test,labels_copy_train,labels_copy_test = train_test_split(features, labels_copy, random_state=3, shuffle=True)

In [35]:
# Building models
clf1 = SVC(C=100)
clf1.fit(features_train, labels_train)
clf1.score(features_test, labels_test)

0.61006289308176098

In [33]:
clf2 = RandomForestClassifier(n_estimators=100, min_samples_split=10)
clf2.fit(features_train, labels_train)
clf2.score(features_test, labels_test)

0.62893081761006286

In [105]:
# Building neural network
model = Sequential()
model.add(Dense(100, input_dim=ndim, activation="relu"))
model.add(Dense(100,activation="relu"))
model.add(Dense(75, activation="relu"))
model.add(Dense(14, activation="softmax"))
model.compile(optimizer="adam", loss=keras.losses.mse, metrics=["accuracy"])

In [106]:
model.fit(features_copy_train, labels_copy_train,epochs=200, batch_size=100, validation_data=(features_copy_test, labels_copy_test) )

Train on 477 samples, validate on 159 samples
Epoch 1/200
477/477 [==============================] - 0s 741us/step - loss: 0.0668 - acc: 0.1426 - val_loss: 0.0657 - val_acc: 0.1384
Epoch 2/200
477/477 [==============================] - 0s 38us/step - loss: 0.0645 - acc: 0.1887 - val_loss: 0.0650 - val_acc: 0.1887
Epoch 3/200
477/477 [==============================] - 0s 41us/step - loss: 0.0631 - acc: 0.2285 - val_loss: 0.0641 - val_acc: 0.2201
Epoch 4/200
477/477 [==============================] - 0s 39us/step - loss: 0.0621 - acc: 0.2600 - val_loss: 0.0635 - val_acc: 0.2201
Epoch 5/200
477/477 [==============================] - 0s 43us/step - loss: 0.0614 - acc: 0.2683 - val_loss: 0.0629 - val_acc: 0.2138
Epoch 6/200
477/477 [==============================] - 0s 48us/step - loss: 0.0607 - acc: 0.2767 - val_loss: 0.0624 - val_acc: 0.2264
Epoch 7/200
477/477 [==============================] - 0s 36us/step - loss: 0.0601 - acc: 0.2809 - val_loss: 0.0620 - val_acc: 0.2516
Epoch 8/200
477

477/477 [==============================] - 0s 44us/step - loss: 0.0330 - acc: 0.7002 - val_loss: 0.0496 - val_acc: 0.4906
Epoch 122/200
477/477 [==============================] - 0s 38us/step - loss: 0.0333 - acc: 0.6792 - val_loss: 0.0507 - val_acc: 0.4717
Epoch 123/200
477/477 [==============================] - 0s 49us/step - loss: 0.0329 - acc: 0.6918 - val_loss: 0.0488 - val_acc: 0.5409
Epoch 124/200
477/477 [==============================] - 0s 41us/step - loss: 0.0330 - acc: 0.6960 - val_loss: 0.0497 - val_acc: 0.4969
Epoch 125/200
477/477 [==============================] - 0s 45us/step - loss: 0.0331 - acc: 0.6918 - val_loss: 0.0497 - val_acc: 0.5094
Epoch 126/200
477/477 [==============================] - 0s 48us/step - loss: 0.0327 - acc: 0.7107 - val_loss: 0.0489 - val_acc: 0.5346
Epoch 127/200
477/477 [==============================] - 0s 45us/step - loss: 0.0329 - acc: 0.6939 - val_loss: 0.0499 - val_acc: 0.5157
Epoch 128/200
477/477 [==============================] - 0s 47

## The model is overfitting.

In [46]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_features_train = scaler.fit_transform(features_copy_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [47]:
scaled_features_train

array([[ 0.40455895, -1.44032478, -1.397596  , ..., -0.04583492,
        -0.57977104, -0.58299883],
       [-0.51436781,  1.08902605,  1.04323985, ..., -0.04583492,
        -0.57977104, -0.58299883],
       [ 0.40455895, -1.44032478,  0.43303089, ..., -0.04583492,
        -0.57977104, -0.58299883],
       ..., 
       [ 0.40455895,  0.14051949,  0.12792641, ..., -0.04583492,
        -0.57977104,  1.7152693 ],
       [-1.43329456,  0.7728572 ,  0.73813537, ..., -0.04583492,
        -0.57977104, -0.58299883],
       [ 1.93610354,  0.14051949,  0.12792641, ..., -0.04583492,
        -0.57977104, -0.58299883]])

In [48]:
model.fit(scaled_features_train, labels_copy_train, epochs=100, batch_size=100)

Epoch 1/100
477/477 [==============================] - 0s 37us/step - loss: 11.1452 - acc: 0.0797
Epoch 2/100
477/477 [==============================] - 0s 34us/step - loss: 7.4658 - acc: 0.1342
Epoch 3/100
477/477 [==============================] - 0s 33us/step - loss: 5.0444 - acc: 0.2348
Epoch 4/100
477/477 [==============================] - 0s 35us/step - loss: 3.9800 - acc: 0.2704
Epoch 5/100
477/477 [==============================] - 0s 33us/step - loss: 3.4181 - acc: 0.2893
Epoch 6/100
477/477 [==============================] - 0s 34us/step - loss: 3.0252 - acc: 0.3187
Epoch 7/100
477/477 [==============================] - 0s 33us/step - loss: 2.6963 - acc: 0.3333
Epoch 8/100
477/477 [==============================] - 0s 35us/step - loss: 2.5453 - acc: 0.3249
Epoch 9/100
477/477 [==============================] - 0s 34us/step - loss: 2.3404 - acc: 0.3690
Epoch 10/100
477/477 [==============================] - 0s 36us/step - loss: 2.1892 - acc: 0.4067
Epoch 11/100
477/477 [======

In [56]:
model.evaluate(features_copy_test, labels_copy_test)

159/159 [==============================] - 0s 74us/step


[15.366900869885331, 0.037735848869167782]

In [51]:
scaled_features_test = scaler.fit_transform(features_copy_test)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [58]:
from keras import backend as K

In [59]:
def new_relu(x):
    return K.relu(x, max_value=13)
    

In [61]:
nn = Sequential()
nn.add(Dense(100, activation="relu", input_dim=features_train.shape[1]))
nn.add(Dense(100, activation="relu"))
nn.add(Dense(75, activation="relu"))
nn.add(Dense(1, activation=new_relu))

In [62]:
nn.compile(optimizer="adam", loss=keras.losses.mse, metrics=["accuracy"])


In [65]:
nn.fit(features_train, labels_train, epochs=150)

Epoch 1/150
477/477 [==============================] - 0s 69us/step - loss: 2.1320 - acc: 0.4990
Epoch 2/150
477/477 [==============================] - 0s 66us/step - loss: 2.1370 - acc: 0.5052
Epoch 3/150
477/477 [==============================] - 0s 75us/step - loss: 2.3645 - acc: 0.4172
Epoch 4/150
477/477 [==============================] - 0s 75us/step - loss: 2.2300 - acc: 0.4319
Epoch 5/150
477/477 [==============================] - 0s 84us/step - loss: 2.1100 - acc: 0.5283
Epoch 6/150
477/477 [==============================] - 0s 75us/step - loss: 2.0447 - acc: 0.5556
Epoch 7/150
477/477 [==============================] - 0s 76us/step - loss: 2.0964 - acc: 0.4990
Epoch 8/150
477/477 [==============================] - 0s 76us/step - loss: 2.0494 - acc: 0.5346
Epoch 9/150
477/477 [==============================] - 0s 77us/step - loss: 2.0787 - acc: 0.5514
Epoch 10/150
477/477 [==============================] - 0s 72us/step - loss: 2.0113 - acc: 0.5660
Epoch 11/150
477/477 [=======

In [66]:
nn.evaluate(features_test, labels_test)

159/159 [==============================] - 0s 325us/step


[3.9925836886999742, 0.49056603511174518]

In [77]:
pred = np.round(nn.predict(features_test))

In [74]:
features_test[7]

array([9, 7, 7, 0, 1, 1, 0])

In [78]:
pred[7]

array([ 9.], dtype=float32)

## By using a custom activation function, we are getting an accuracy of 50% on the test data, hence preventing overfitting.

In [79]:
nn

In [101]:
model.evaluate(features_copy_test, labels_copy_test)

159/159 [==============================] - 0s 73us/step


[1.6023147436057996, 0.50314465127651042]

In [84]:
pred = np.argmax(model.predict(features_copy_test), axis=1)

In [85]:
pred

array([ 5,  5,  6,  7,  9,  7,  8,  9,  9, 10,  1,  1,  9,  6,  9,  1,  1,
        4,  2,  1,  5,  1,  2,  4,  9,  8, 12,  4,  0,  4,  2,  6,  7,  9,
        4,  8,  7,  9,  5,  8,  2,  1, 10,  9, 10,  8,  0,  6,  4,  9,  7,
        9,  1,  9,  0,  6,  4,  7,  0,  8,  4,  1,  5,  1,  9,  4,  9,  4,
        1,  6,  8,  2, 12,  9,  4,  9,  1,  9,  8, 10,  9,  0,  8,  4,  7,
        6,  1,  5,  6,  4,  5,  6,  1,  1,  8,  4,  4,  1,  9,  9, 10,  5,
        9,  5,  8,  9, 12,  9,  8,  8,  4,  8,  1, 10,  0,  5,  9, 10,  8,
        1,  1,  5,  8,  1,  9, 12,  5,  4,  0, 10,  1,  9,  1,  8,  4,  5,
        6, 10,  8, 10,  6,  9,  1,  3,  6,  6, 10,  9,  7,  4, 12,  8,  0,
        5,  7,  4, 12,  7,  4])

In [88]:
features_copy_test[0]


array([7, 5, 7, 1, 1, 0, 0])

In [90]:
pred[0]

5

In [97]:
np.argmax(labels_copy_test[1], axis=0)

5

# With nn, we get 49% and with model we get 55% using one hot encoding.

In [104]:
nn.save("nn-matches-model-with-49%.hdf5")
model.save('model-matches-model-with-55%.hdf5')

In [107]:
model.evaluate(features_copy_test, labels_copy_test)

159/159 [==============================] - 0s 67us/step


[0.048274208164814883, 0.52201257561737635]

In [112]:
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier

In [113]:
clf = ExtraTreesClassifier(n_estimators=100)
clf.fit(features_train, labels_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [114]:
clf.score(features_test, labels_test)

0.61635220125786161

In [116]:
p = clf.predict(features_test)

In [117]:
features_test[0]

array([7, 5, 7, 1, 1, 0, 0])

In [118]:
a = np.array([5, 7, 5, 1, 1, 0, 0])

In [119]:
clf.predict(a.reshape(1,-1))

array([7])

In [130]:
np.round(model.predict(a.reshape(1,-1))

SyntaxError: unexpected EOF while parsing (<ipython-input-130-05bf4e0d4c51>, line 1)

In [132]:
c = model.predict(a.reshape(1,-1))

In [135]:
np.argmax(c, axis=1)

array([7])

In [138]:
nn.predict(a.reshape(1,-1)).astype(int)

array([[5]])

In [148]:
clf.predict(features_test)

array([ 5,  5,  6,  7,  8,  7,  8,  9,  1,  9,  1,  1,  9,  6,  7,  1,  1,
        4,  2,  1,  5,  1,  2,  4,  0,  9, 12,  4,  0,  4,  2,  6,  6,  6,
        4,  8,  7,  8,  5,  8,  3,  1, 10,  8, 10,  8,  0,  9,  4,  7,  7,
        8,  1,  7,  0,  6,  4,  7,  0,  5,  4,  1,  5,  1, 10,  3,  9,  4,
        1,  5,  7,  1, 12,  9,  4,  8,  1,  8,  8,  9,  6,  0,  8,  4,  7,
        6,  1,  5,  6,  4,  5,  9,  1,  1,  8,  4,  4,  1,  8,  9, 10,  5,
       10,  6,  8,  9,  9,  8,  8,  8,  4,  7,  1,  9,  9,  5,  8, 10,  8,
        1,  1,  5,  7,  1,  9, 12,  5,  4,  1, 10,  1,  7,  1,  8,  5,  5,
        7, 11,  8,  8,  6,  9,  1,  3,  7,  6, 11,  9,  7,  4, 12,  8,  0,
        5,  7,  5, 12,  8,  4])

In [149]:
labels_test

array([ 7,  5,  6,  7,  9,  1,  8,  9,  1,  9,  9,  1, 13,  6, 10,  1,  1,
        4,  2,  1,  5,  6,  6,  4,  9,  7, 10,  4,  0,  4,  2,  4,  1,  6,
        4,  8,  6, 10,  5,  8,  3,  1, 10,  8,  9,  6,  0, 12,  4,  9,  6,
        9,  1,  9,  0,  6,  4,  7,  0,  9,  4,  1,  5,  1,  7,  3,  8,  4,
        1,  5,  8,  4, 12,  9,  6,  8,  6,  9,  9,  9,  9,  0,  8,  4,  7,
        6,  7,  5,  6,  0,  0,  9,  9,  1,  9,  8,  0,  1,  8,  9, 10,  5,
       10,  6,  8,  9, 12,  8,  8,  8,  4,  7,  1, 12, 13,  5,  9,  1,  4,
        1,  3,  5,  8,  1,  9, 12,  7,  5,  0, 10,  4, 10,  6,  8,  4,  5,
        7, 11,  8, 10,  6,  9,  1,  3,  6,  6, 11,  5,  1,  4, 10,  8,  0,
        5, 10,  4, 12,  8,  4])

In [150]:
labels_test

array([ 7,  5,  6,  7,  9,  1,  8,  9,  1,  9,  9,  1, 13,  6, 10,  1,  1,
        4,  2,  1,  5,  6,  6,  4,  9,  7, 10,  4,  0,  4,  2,  4,  1,  6,
        4,  8,  6, 10,  5,  8,  3,  1, 10,  8,  9,  6,  0, 12,  4,  9,  6,
        9,  1,  9,  0,  6,  4,  7,  0,  9,  4,  1,  5,  1,  7,  3,  8,  4,
        1,  5,  8,  4, 12,  9,  6,  8,  6,  9,  9,  9,  9,  0,  8,  4,  7,
        6,  7,  5,  6,  0,  0,  9,  9,  1,  9,  8,  0,  1,  8,  9, 10,  5,
       10,  6,  8,  9, 12,  8,  8,  8,  4,  7,  1, 12, 13,  5,  9,  1,  4,
        1,  3,  5,  8,  1,  9, 12,  7,  5,  0, 10,  4, 10,  6,  8,  4,  5,
        7, 11,  8, 10,  6,  9,  1,  3,  6,  6, 11,  5,  1,  4, 10,  8,  0,
        5, 10,  4, 12,  8,  4])

In [152]:
features[0]

array([0, 4, 4, 0, 1, 1, 0])

In [153]:
labels[0]

0

In [155]:
a = model.predict(features[0].reshape(1,-1))

In [156]:
a

array([[  9.81611907e-01,   8.42660107e-03,   9.77830013e-08,
          8.47694930e-03,   1.00995225e-04,   1.28758221e-03,
          5.53907121e-05,   3.99190758e-05,   4.88160254e-07,
          3.05353232e-09,   1.57500253e-08,   9.04296105e-09,
          5.01230311e-08,   2.48539900e-09]], dtype=float32)

In [157]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 100)               800       
_________________________________________________________________
dense_14 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_15 (Dense)             (None, 75)                7575      
_________________________________________________________________
dense_16 (Dense)             (None, 14)                1064      
Total params: 19,539
Trainable params: 19,539
Non-trainable params: 0
_________________________________________________________________


In [158]:
np.argmax(a, axis=1)

array([0])